In [3]:
import flowcode
import processing
import res_flow_vis as visual
import device_use

import torch
import numpy as np
import time

In [4]:
filename = "TEST"

In [5]:
#Initiate processing
mp = processing.Processor()

In [6]:
#Rewrite the code to fit the new Processor Code in processing.py

In [7]:
#Get data raw data
Data = mp.get_data("Old/g8.26e11.npy")

In [8]:
#Clean data
Data_const = mp.constrain_data(Data)

In [9]:
#Right device
device = device_use.device_use

In [10]:
#Instantiate the model
model = flowcode.NSFlow(8, 10, 0, flowcode.NSF_CL2, K=8, B=3, network = flowcode.MLP, network_args=(16,4,0.2))
model = model.to(device)

In [11]:
Data_flow = mp.Data_to_flow(Data_const)

In [10]:
#Save relevant data to the drive for external python file (device needs to be GPU) to do the training in background...
torch.save(Data_flow, "NC_trainer/data_NC_trainer.pth")
torch.save(model, "NC_trainer/model_NC_trainer.pth")
np.save("NC_trainer/params_NC_trainer.npy", np.array([150,0.0003,1024,0.999]))
np.save("NC_trainer/filename_NC_trainer.npy", filename)

In [12]:
#Train the model
#list to collect loss into
my_loss_saver = []
start = time.perf_counter()
flowcode.train_flow(model, Data_flow, np.array([]), 100, lr=0.016, batch_size=1024, loss_saver=my_loss_saver, gamma=0.998)
end = time.perf_counter()
torch.save(model.state_dict(), f"saves/{filename}.pth")
np.save(f"saves/loss_{filename}.npy",np.array(my_loss_saver+[end-start]))

Step 0 of 71402, Loss:16.726869583129883, lr=0.016
Step 100 of 71402, Loss:7.617042808532715, lr=0.01568286469363119
Step 200 of 71402, Loss:5.122779378890991, lr=0.01537201531242148
Step 300 of 71402, Loss:4.765067081451416, lr=0.015067327263320803
Step 400 of 71402, Loss:4.304386401176453, lr=0.01476867842283253


KeyboardInterrupt: 

In [ ]:
#torch.save(model.state_dict(), f"saves/{filename}.pth")
#np.save(f"saves/loss_{filename}.npy",np.array(my_loss_saver+[358*60]))

In [ ]:
#Load in training results:
model.load_state_dict(torch.load(f"saves/{filename}.pth"))
loss_results = np.load(f"saves/loss_{filename}.npy")
loss_results, tot_time = loss_results[:-1], loss_results[-1]/60

In [ ]:
#Sample model
N_samples = Data_const.shape[0]
model.eval()
with torch.inference_mode():
    res = (model.sample_Flow(N_samples, torch.tensor([]))).cpu()

flow_sample = res
flow_sample = mp.sample_to_Data(flow_sample)

In [ ]:
#Get result plots
visual.get_result_plots(Data_const, flow_sample, label=filename)

In [ ]:
#Get loss curve
visual.loss_plot(loss_results, tot_time=tot_time, savefig=filename)